# Importing important libraries

## Steps to be followed

1. Importing nessesary ibraries
2. Creating S3 bucket
3. Mapping trin and test data in S3
4. Mapping the pat od the models in S3

In [17]:
pip install sagemaker

Note: you may need to restart the kernel to use updated packages.


In [19]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [20]:
bucket_name = 'prueba2512' #we create the bucket name
my_region = boto3.session.Session().region_name #set the region of the instance
print("my current region is ",my_region)

my current region is  us-east-1


In [21]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error',e)

S3 bucket created successfully


In [22]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path = 's3://{}/{}/output'.format(bucket_name,prefix) #output folder where to store my model
print(output_path)

s3://prueba2512/xgboost-as-a-built-in-algo/output


### Downloading the dataset and storing in S3

In [23]:
import pandas as pd
import urllib 
try: 
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print("Success: downloaded bank_clean.csv.")
except Exception as e:
    print("Data load error: ",e)
                              
try:
    model_data = pd.read_csv('./bank_clean.csv', index_col=0)
    print("Success: Data loaded into dataframe")
except Exception as e:
    print("Data load error: ",e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe


In [24]:
## train test split
import numpy as np
train_data, test_data = np.split(model_data.sample(frac= 1, random_state=1729), [int(0.7*len(model_data))])

print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [25]:
### In the documentation of AWS Sagemaker says that the dependence feature has to be first in the dataset
## we move the dependence feature to the first place and savinf train and test into buckets
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no','y_yes'],
                                                 axis=1)],axis=1).to_csv('train.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data ='s3://{}/{}/train'.format(bucket_name,prefix), content_type = 'csv' ) #the package change


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [26]:
#now for training
pd.concat([test_data['y_yes'], test_data.drop(['y_no','y_yes'],
                                                 axis=1)],axis=1).to_csv('test.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data ='s3://{}/{}/test'.format(bucket_name,prefix), content_type = 'csv' ) #the package change


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


### Building models Xgboost-inbuilt Algorithm

In [28]:
#this line automatically looks for the 
container = get_image_uri(boto3.Session().region_name,
                         'xgboost',
                         repo_version='1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [33]:
#initialize hyperparameters
hyperparameters = {
    'max_depth': "5",
    "eta": "0.2",
    "gamma": '4',
    'min_child_weight':'6',
    'subsample':'0.7',
    'num_round':'100'
}

In [34]:
#contruct a sagemaker estimator that call the xgboost-container
estimator = sagemaker.estimator.Estimator(image_name = container,
                                         hyperparameters=hyperparameters,
                                         role= sagemaker.get_execution_role(),
                                         train_instance_count = 1,
                                         train_instance_type = 'ml.m5.2xlarge', 
                                         train_volume_size = 5, #5 GB
                                         output_path=output_path,
                                         train_use_spot_instances = True,
                                         train_max_run = 300,
                                         train_max_wait = 600 
                                         )


Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [35]:
estimator.fit({'train':s3_input_train,'validation':s3_input_test})



2022-03-13 01:41:34 Starting - Starting the training job...
2022-03-13 01:41:38 Starting - Launching requested ML instances.........
2022-03-13 01:43:07 Starting - Preparing the instances for training...
2022-03-13 01:43:51 Downloading - Downloading input data
2022-03-13 01:43:51 Training - Downloading the training image.......INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[01:45:07] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of CSV input is ','
[01:45:07] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/validation?format=csv&lab

### Deploy Machine Learning Model

In [36]:
xgb_predictor = estimator.deploy(initial_instance_count = 1,instance_type= 'ml.m5.2xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----!

#### Prediction of the test data

In [39]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no','y_yes'],axis = 1).values
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:],sep = ',')
print(predictions_array.shape)

(12357,)


In [40]:
predictions_array

array([0.05987549, 0.05381063, 0.04968706, ..., 0.04968706, 0.04968706,
       0.03934947])

In [41]:
cm = pd.crosstab(index=test_data['y_yes'], columns = np.round(predictions_array),rownames=['observed'],
                colnames = ['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0];tp =cm.iloc[1,1]; fp=cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn) *100

print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")

print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))



Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10824)    30% (112)
Purchase        10% (1165)     70% (256) 



# deleting the endpoints


In [43]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '9K7P43HXDXFJQ13W',
   'HostId': 'VUD/k33GTAQmtv1W2K/zQU2xun6IJObG64ZpEIWfgHdzi30as9Ug9QU5J3X/PHuiEGD2SBchD24=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'VUD/k33GTAQmtv1W2K/zQU2xun6IJObG64ZpEIWfgHdzi30as9Ug9QU5J3X/PHuiEGD2SBchD24=',
    'x-amz-request-id': '9K7P43HXDXFJQ13W',
    'date': 'Sun, 13 Mar 2022 02:09:59 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/train/train.csv'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-03-13-01-41-34-515/output/model.tar.gz'},
   {'Key': 'xgboost-as-a-built-in-algo/test/test.csv'}]}]